In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 1year.arff to 1year.arff
User uploaded file "1year.arff" with length 3432892 bytes


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from scipy.io import arff
from pandas_profiling import ProfileReport
data = arff.loadarff('1year.arff')
data=pd.DataFrame(data[0])

In [ ]:
df = pd.DataFrame(data)

print(df.info())

In [ ]:
df.describe()

In [ ]:
!pip uninstall pandas-profiling
!pip install pandas-profiling[notebook]
!pip show pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.9/345.9 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.6 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py

Name: pandas-profiling
Version: 3.6.6
Summary: Deprecated 'pandas-profiling' package, use 'ydata-profiling' instead
Home-page: https://github.com/ydataai/pandas-profiling
Author: YData Labs Inc
Author-email: opensource@ydata.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: ydata-profiling
Required-by: 


In [ ]:
prof = ProfileReport(df)
prof.to_file("report.html")

In [ ]:
files.download('report.html')

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = df.drop('class', axis=1)
y = df['class'].astype(int)
imputed_X = imputer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imputed_X, y, test_size=0.2, random_state=36)
print("Training dataset size", X_train.shape, y_train.shape)
print("Testing dataset size:", X_test.shape, y_test.shape)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
max_depth_list = np.linspace(1, 15, 10, dtype=int)
max_leaf_nodes_list = np.linspace(2, 120, 10, dtype=int)
alpha = np.linspace(0.0001, 0.01, 10)

In [ ]:
grid_params = { 'max_depth':max_depth_list, 'max_leaf_nodes':max_leaf_nodes_list, 'ccp_alpha':alpha}
grid_params

In [ ]:
classifier = DecisionTreeClassifier(random_state=36)
cross_valid = GridSearchCV(estimator=classifier, param_grid=grid_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
cross_valid.fit(X_train, y_train)

In [ ]:
optimal_params = cross_valid.best_params_
optimal_params

##3.	What is the performance of the optimal decision tree? Comment on the accuracy and confusion matrix. What are the importance features?

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
best_classifier = cross_valid.best_estimator_
y_pred = best_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

In [ ]:
feat_imp = best_classifier.feature_importances_
print("Feature Importances:\n", feat_imp)

In [ ]:
importances_with_features = sorted(zip(feat_imp, X.columns))
for importance, feature in importances_with_features:
  if(importance!=0):
    print(f"{feature}: {importance}")
print('\nAll other parameters have 0 importance')

###4.	Use 5 fold cross validation to tune the following parameters: n_estimators, and max_depth for a bagging tree. Again, you can use 10 candidate values for each parameter. 

In [ ]:
from sklearn.ensemble import BaggingClassifier
n_estimators_list = np.linspace(10, 120, 10, dtype=int)
max_depth_list = np.linspace(1, 25, 10, dtype=int)

In [ ]:
grid_params = {
    'base_estimator__max_depth': max_depth_list,
    'n_estimators': n_estimators_list
}

In [ ]:
estimator = DecisionTreeClassifier(random_state=36)
bagging_classifier = BaggingClassifier(base_estimator=estimator, random_state=36)
cross_valid = GridSearchCV(estimator=bagging_classifier, param_grid=grid_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
cross_valid.fit(X_train, y_train)

In [ ]:
best_params = cross_valid.best_params_
print("Best parameters:", best_params)

In [ ]:
best_classifier = cross_valid.best_estimator_
y_pred = best_classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", c_matrix)

In [ ]:
imp = np.mean([tree.feature_importances_ for tree in best_classifier.estimators_], axis=0)
print("Feature Importances:\n", imp)

In [ ]:
importances_with_features = sorted(zip(imp, X.columns))
for importance, feature in importances_with_features:
  if(importance!=0):
    print(f"{feature}: {importance}")
print('\nAll other parameters have 0 importance')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rate_list = np.logspace(-3, -2, 10)
n_estimators_list = np.linspace(10, 150, 10, dtype=int)
param_grid = {
    'learning_rate': learning_rate_list,
    'n_estimators': n_estimators_list
}

In [ ]:
gradboost_classifier = GradientBoostingClassifier(random_state=36)
cross_valid = GridSearchCV(estimator=gradboost_classifier, param_grid=grid_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
cross_valid.fit(X_train, y_train)

In [ ]:
cross_valid.fit(X_train, y_train)

In [ ]:
best_params = cross_valid.best_params_
print("Best parameters:", best_params)

In [ ]:
best_gb_classifier = cross_valid.best_estimator_
y_pred = best_classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", c_matrix)

In [ ]:
imp = best_gb_classifier.feature_importances_
print("Feature Importances:\n", imp)

In [ ]:
importances_with_features = sorted(zip(imp, X.columns))
for importance, feature in importances_with_features:
  if(importance!=0):
    print(f"{feature}: {importance}")
print('\nAll other parameters have 0 importance')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_features_list = ['auto', 'sqrt', 'log2']
n_estimators_list = np.linspace(10, 150, 10, dtype=int)
param_grid = {
    'max_features': max_features_list,
    'n_estimators': n_estimators_list
}

In [ ]:
randforest_classifier = RandomForestClassifier(random_state=36)
cross_valid = GridSearchCV(estimator=randforest_classifier, param_grid=grid_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
cross_valid.fit(X_train, y_train)

In [ ]:
cross_valid.fit(X_train, y_train)

In [ ]:
best_params = cross_valid.best_params_
print("Best parameters:", best_params)

In [ ]:
best_rf_classifier = cross_valid.best_estimator_
y_pred = best_classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", c_matrix)

In [ ]:
imp = best_rf_classifier.feature_importances_
print("Feature Importances:\n", imp)

In [ ]:
importances_with_features = sorted(zip(imp, X.columns))
for importance, feature in importances_with_features:
  if(importance!=0):
    print(f"{feature}: {importance}")
print('\nAll other parameters have 0 importance')